In [10]:
!docker stop ipfs_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [11]:
!docker rm ipf_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [12]:
!docker run -d --name ipfs_host2 -v $ipfs_staging:/export -v $ipfs_data:/data/ipfs -p 4001:4001 -p 4001:4001/udp -p 127.0.0.1:8080:8080 -p 127.0.0.1:5001:5001 ipfs/go-ipfs:latest

539f3618e5ff0b0a85e7d61b47c4aeeb89876547f596fe3bad72779d35980a25


In [1]:
!mkdir /tmp/posts
!mkdir /tmp/comments

In [2]:
%load_ext autoreload
%autoreload 2

In [75]:
import json
import sqlite3
import pandas as pd
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException
from substrate_helpers import make_call, addSchema, get_msa_id, \
            get_signature, create_msa_with_delegator, mint_votes, mint_user, get_schemas_from_pattern, \
            get_content_from_schemas, get_user, follow_user, add_delegate, mint_data

In [69]:
path = '/tmp/'
bob = Keypair.create_from_uri('//Bob')
substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [24]:
bob_msa_id = get_msa_id(bob, create=True)

In [25]:
bob_msa_id

1

In [76]:
con = sqlite3.connect('postthreadV1_write.db')
cur = con.cursor()

# Make schemas

In [95]:
post_schema = "category STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC"
post_schemaId = addSchema(post_schema, wait_for_inclusion=True, wait_for_finalization=False)

comment_schema = "post_hash STRING,parent_hash STRING,depth NUMERIC,body STRING"
comment_schemaId = addSchema(comment_schema, wait_for_inclusion=True, wait_for_finalization=False)

vote_schema = "post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC"
vote_schemaId = addSchema(vote_schema, wait_for_inclusion=True, wait_for_finalization=False)

user_schema = "msa_id NUMERIC,username STRING,password STRING,profile_pic STRING,wallet_ss58_address STRING"
user_schemaId = addSchema(user_schema, wait_for_inclusion=True, wait_for_finalization=False)

follow_schema = "protagonist_msa_id NUMERIC,antagonist_msa_id NUMERIC,event STRING"
follow_schemaId = addSchema(follow_schema, wait_for_inclusion=True, wait_for_finalization=False)

link_schema = "account_type STRING,account_value STRING"
link_schemaId = addSchema(link_schema, wait_for_inclusion=True, wait_for_finalization=False)

payout_schema = "payout_amount NUMERIC"
payout_schemaId = addSchema(payout_schema, wait_for_inclusion=True, wait_for_finalization=False)

In [96]:
schemas = {
    "post": post_schemaId, "comment": comment_schemaId, 
    "vote": vote_schemaId, "user": user_schemaId, "follow": follow_schemaId, 
    "link": link_schemaId, "payout": payout_schemaId,
}
json.dump(schemas, open("schemas.json", "w"))

In [97]:
schemas

{'post': 7,
 'comment': 8,
 'vote': 3,
 'user': 4,
 'follow': 5,
 'link': 12,
 'payout': 11}

# Mint accounts 

In [87]:
accounts = {}
for account in ['Alice', 'Charlie', 'Dave', 'Eve', 'Ferdie']:
    wallet = Keypair.create_from_uri(f'//{account}password')
    user_msa_id = create_msa_with_delegator(bob, wallet)
    receipt_user = mint_user(user_msa_id, account, "password", "profile_pic", wallet)
    accounts[account] = user_msa_id

In [88]:
json.dump(accounts, open("accounts.json", "w"))

In [89]:
accounts

{'Alice': 693, 'Charlie': 382, 'Dave': 694, 'Eve': 695, 'Ferdie': 696}

# Have accounts follow each other

In [64]:
for name1, k1 in accounts.items():
    for name2, k2 in accounts.items():
        if k1 != k2:
            print(name1, name2)
            receipt = follow_user(k1, k2, True, wait_for_inclusion=False, wait_for_finalization=False)

Alice Charlie
Alice Dave
Alice Eve
Alice Ferdie
Charlie Alice
Charlie Dave
Charlie Eve
Charlie Ferdie
Dave Alice
Dave Charlie
Dave Eve
Dave Ferdie
Eve Alice
Eve Charlie
Eve Dave
Eve Ferdie
Ferdie Alice
Ferdie Charlie
Ferdie Dave
Ferdie Eve


In [65]:
receipt = follow_user(accounts['Dave'], accounts['Eve'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(accounts['Eve'], accounts['Dave'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(accounts['Dave'], accounts['Eve'], True, wait_for_inclusion=False, wait_for_finalization=False)

# Test post

In [70]:
post_data = {
    "category": "test",
    "title": "test title",
    "body": "test post",
    "url": "",
    "is_nsfw": False
}

username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

post_data_hash, receipt_post = mint_data(post_data, user_msa_id, schemas['post'], path+'posts/', 
                                         wait_for_inclusion=False, wait_for_finalization=False)

In [ ]:
post_data = {
    "category": "test",
    "title": "test title",
    "body": "test post",
    "url": "",
    "is_nsfw": False
}

username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

post_data_hash, receipt_post = mint_data(post_data, user_msa_id, schemas['post'], path+'posts/', 
                                         wait_for_inclusion=False, wait_for_finalization=False)

In [78]:
example_post = pd.read_sql_query(f"SELECT ipfs_hash FROM post WHERE msa_id_from_query = {accounts['Charlie']} LIMIT 1", con)['ipfs_hash'].iloc[0]
example_post

'QmbG2dxLWHKvdgsTqDzLjhD96BXrLQUeJkrLuasmPZKgn9'

In [80]:
comment_data = {
    "post_hash": example_post,
    "parent_hash": example_post,
    "depth": 0,
    "body": "example comment 2",
}
username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, schemas['comment'], path+'comments/', 
                                               wait_for_inclusion=True, wait_for_finalization=False)


In [101]:
account_type = "gmail"
account_value = "example@gmail.com"
link_data = '{' + f'"account_type": "{account_type}","account_value": "{account_value}"' + '}'

username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

link_data_hash, receipt_link = mint_data(link_data, user_msa_id, schemas['link'], 
                                               wait_for_inclusion=True, wait_for_finalization=False)

In [102]:
receipt_link.error_message